# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import re
import sqlalchemy as sql
import pickle
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import f1_score, recall_score, precision_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
# load data from database
engine = sql.create_engine('sqlite:///rvd-project_disaster_response_pipelines.db')
df = pd.read_sql_table("messages",con=engine)

# Defining the columns to get Y
y_columns = list(df.columns)
y_columns.remove('id')
y_columns.remove('message')
y_columns.remove('original')
y_columns.remove('genre')

Y = df[y_columns].values
X = df['message'].values

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    tokens = nltk.tokenize.word_tokenize(text)
    lemmatizer = nltk.stem.WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return(clean_tokens)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf',MultiOutputClassifier(RandomForestClassifier(max_depth=7)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [6]:
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
# Predicting Results
y_pred_train = pipeline.predict(X_train)
y_pred = pipeline.predict(X_test)

In [8]:
for i in range(0,len(y_columns)):
    print( '------------------------------------------' )
    print( y_columns[i] )
    print( classification_report(y_test[:,i], y_pred[:,i], target_names=['class 0 - false','class 1 - true ','class 2 - ???? ']) )

------------------------------------------
related
                 precision    recall  f1-score   support

class 0 - false       1.00      0.01      0.02       801
class 1 - true        0.66      1.00      0.79      1579
class 2 - ????        0.00      0.00      0.00        28

    avg / total       0.76      0.66      0.53      2408

------------------------------------------
request
                 precision    recall  f1-score   support

class 0 - false       0.70      0.99      0.82      1569
class 1 - true        0.93      0.21      0.34       839

    avg / total       0.78      0.72      0.65      2408

------------------------------------------
offer
                 precision    recall  f1-score   support

class 0 - false       1.00      1.00      1.00      2405
class 1 - true        0.00      0.00      0.00         3

    avg / total       1.00      1.00      1.00      2408

------------------------------------------
aid_related
                 precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 3
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 1, does not match size of target_names, 3
  .format(len(labels), len(target_names))


### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
parameters = {
        #'vect__ngram_range': ((1, 1), (1, 2)),
        #'vect__max_df': (0.5, 0.75, 1.0),
        #'vect__max_features': (None, 5000, 10000),
        'tfidf__use_idf': (True, False),
        'clf__estimator': (
            RandomForestClassifier(max_depth= 3),
            RandomForestClassifier(max_depth= 5),
            RandomForestClassifier(max_depth= 7),
            RandomForestClassifier(max_depth=10)
        )
    }

cv = GridSearchCV(pipeline, param_grid=parameters)

cv.fit(X_train,y_train)
cv.best_params_

{'clf__estimator': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=7, max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
             oob_score=False, random_state=None, verbose=0,
             warm_start=False), 'tfidf__use_idf': False}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [10]:
y_pred = cv.best_estimator_.predict(X_test)

for i in range(0,len(y_columns)):
    print( '------------------------------------------' )
    print( y_columns[i] )
    print( classification_report(y_test[:,i], y_pred[:,i], target_names=['class 0 - false','class 1 - true ','class 2 - ???? ']) )

------------------------------------------
related
                 precision    recall  f1-score   support

class 0 - false       0.79      0.06      0.12       801
class 1 - true        0.67      0.99      0.80      1579
class 2 - ????        0.00      0.00      0.00        28

    avg / total       0.70      0.67      0.56      2408

------------------------------------------
request
                 precision    recall  f1-score   support

class 0 - false       0.71      0.99      0.82      1569
class 1 - true        0.92      0.23      0.37       839

    avg / total       0.78      0.73      0.67      2408

------------------------------------------
offer
                 precision    recall  f1-score   support

class 0 - false       1.00      1.00      1.00      2405
class 1 - true        0.00      0.00      0.00         3

    avg / total       1.00      1.00      1.00      2408

------------------------------------------
aid_related
                 precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 3
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 1, does not match size of target_names, 3
  .format(len(labels), len(target_names))


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [14]:
parameters = {
        #'vect__ngram_range': ((1, 1), (1, 2)),
        #'vect__max_df': (0.5, 0.75, 1.0),
        #'vect__max_features': (None, 5000, 10000),
        'tfidf__use_idf': (True, False),
        'clf__estimator': (
            RandomForestClassifier(max_depth=7),
            #LogisticRegression(),
            DecisionTreeClassifier(max_depth=7)
        )
    }

cv = GridSearchCV(pipeline, param_grid=parameters)

cv.fit(X_train,y_train)
cv.best_params_

{'clf__estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=7,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, presort=False, random_state=None,
             splitter='best'), 'tfidf__use_idf': True}

In [15]:
y_pred = cv.best_estimator_.predict(X_test)

for i in range(0,len(y_columns)):
    print( '------------------------------------------' )
    print( y_columns[i] )
    print( classification_report(y_test[:,i], y_pred[:,i], target_names=['class 0 - false','class 1 - true ','class 2 - ???? ']) )

------------------------------------------
related
                 precision    recall  f1-score   support

class 0 - false       0.55      0.83      0.66       801
class 1 - true        0.88      0.68      0.77      1579
class 2 - ????        0.00      0.00      0.00        28

    avg / total       0.76      0.72      0.72      2408

------------------------------------------
request
                 precision    recall  f1-score   support

class 0 - false       0.84      0.88      0.86      1569
class 1 - true        0.75      0.69      0.72       839

    avg / total       0.81      0.81      0.81      2408

------------------------------------------
offer
                 precision    recall  f1-score   support

class 0 - false       1.00      1.00      1.00      2405
class 1 - true        0.00      0.00      0.00         3

    avg / total       1.00      1.00      1.00      2408

------------------------------------------
aid_related
                 precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 3
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 1, does not match size of target_names, 3
  .format(len(labels), len(target_names))


### 9. Export your model as a pickle file

In [16]:
filename = 'rvd_model.sav'
pickle.dump(cv.best_estimator_, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.